In [179]:
from langgraph.graph import StateGraph, START, END 
from typing_extensions import TypedDict, Annotated
from typing import Union
from langgraph.types import Send

import operator

In [180]:
class State(TypedDict):
    words: list[str]
    output: Annotated[list[dict[str, Union[str, int]]], operator.add] # For protecting overwrite result in the node_two
    
    
graph_builder = StateGraph(State)

In [181]:
def node_one(state: State):
    print(f"I want to count {len(state["words"])} workds in my state.")
    return {}
    
def node_two(word: str):
    count = len(word)
    return {"output": [{"word": word, "letters": count}]}


In [182]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)

def dispatcher(state: State):
    "Will be making a bunch of node_two"
    return [Send("node_two", word) for word in state["words"]]
    
graph_builder.add_edge(START, "node_one")
graph_builder.add_conditional_edges("node_one", dispatcher)
graph_builder.add_edge("node_two", END)

In [183]:
graph = graph_builder.compile()
print(graph.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| node_one |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [184]:
print(graph.invoke({"words": ["hello", "world", "how", "are", "you", "doing"]}))


I want to count 6 workds in my state.
{'words': ['hello', 'world', 'how', 'are', 'you', 'doing'], 'output': [{'word': 'hello', 'letters': 5}, {'word': 'world', 'letters': 5}, {'word': 'how', 'letters': 3}, {'word': 'are', 'letters': 3}, {'word': 'you', 'letters': 3}, {'word': 'doing', 'letters': 5}]}
